In [ ]:
import numpy as np
import time
import random
import pylab
from hrr import *
%matplotlib inline
from matplotlib import pyplot as plt
import time, sys
from IPython.display import clear_output

In [ ]:
# Number of training cycles
episodes = 50000

# Hrr parameters
hrr_length = 20000
normalized = True

# How many steps to take before quiting
steps_till_quit = 300

signals = ["R", "g"]
goals = [[0, 5], [10, 15]]
goal_store = [[0,0],[5,0],[10,0],[15,0]]

# Maze parameters
size_of_maze = 20
non_obs_task_switch_rate = 500
num_non_obs_tasks = len(goals)
num_obs_tasks = len(signals)

# Arguments for neural network
input_size = hrr_length
output_size = 1
discount = 0.9
alpha = 0.1

# Reward for temporal difference learning
reward_bad = -1
reward_good = 0

# Expolration rate
e_soft = 0.001
rand_on = 1

# Threshold for non observable task switching
threshold_negative = -0.4
treshold_positive = 0.6

# Eligibility trace
eligibility = np.zeros(hrr_length)

# Eligibility trace rate
eli_lambda = 0.01

# Neural network
weights = hrr(hrr_length, normalized)
init_weight = weights
bias = 1

debug = False

In [ ]:
def update_progress(progress, episode):
    bar_length = 50
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Episode {0}, Progress: [{1}] {2:.1f}%".format(episode, "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [ ]:
def get_moves(state, size_of_maze):
    if(state == 0):
        return size_of_maze - 1, 1
    elif(state == size_of_maze - 1):
        return size_of_maze - 2, 0
    else:
        return state - 1, state + 1

In [ ]:
def build_hrr_string(wm, signal, state, atr):
    if wm == "I" and signal == "I":
        return "State:" + str(state) + "*" + "Atr:" + str(atr)
    elif wm == "I":
        return "Signal:" + str(signal) + "*" + "State:" + str(state) + "*" + "Atr:" + str(atr)
    elif signal == "I":
        return "WM:" + str(wm) + "*" + "State:" + str(state) + "*" + "Atr:" + str(atr)
    else:
        return "WM:" + str(wm) + "*" + "Signal:" + str(signal) + "*" + "State:" + str(state) + "*" + "Atr:" + str(atr)

In [ ]:
def context_policy_negative(atr):
    return (atr + 1)%num_non_obs_tasks

In [ ]:
def context_policy_positive(wm, signal, state, atr):
    val = -9999
    temp = -9999
    
    for atr in range(0, num_non_obs_tasks):
        encode_str = build_hrr_string(wm, signal, state, atr)
        temp = np.dot(weights, ltm.encode(encode_str)) + bias
        if temp > val:
            val = temp
            s_atr = atr
            
    return s_atr

In [ ]:
def move_policy(goal, moves, wms, signals, atr, rand_on):
    
    val = -9999
    temp = -9999
    
    for move in moves:
        for wm in list(dict.fromkeys(wms + ["I"])):
            for signal in list(dict.fromkeys(signals + ["I"])):
                if move == goal:
                    encode_str = build_hrr_string(wm, signal, str(move) + "*rewardTkn", atr)
                else:
                    encode_str = build_hrr_string(wm, signal, str(move), atr)
                if (debug):
                    print(encode_str)
                temp = np.dot(weights, ltm.encode(encode_str)) + bias
                if debug:
                    if signal != "I":
                        print("Move: {0}, WM: {1}, Signal: {2}In, Atr: {3}, Value: {4}".format(move, wm, signal, atr, temp))
                    else:
                        print("Move: {0}, WM: {1}, Signal: {2}, Atr: {3}, Value: {4}".format(move, wm, signal, atr, temp))
                if temp > val:
                    val = temp
                    s_move = move
                    if signal != "I":
                        s_wm = signal + "In"
                    else:
                        s_wm = wm
            
    # Random move
    if(np.random.random_sample() < e_soft) and rand_on:
        if(debug):
            print("RANDOM MOVE")
        return (np.random.choice(moves), wm, atr, 1)
    
    return (s_move, s_wm, atr, 0)

In [ ]:
def get_opt_steps(start, goal, size_of_maze):
    opt = abs(goal - start)
    if opt > size_of_maze / 2:
        opt = size_of_maze - opt
    return opt

In [ ]:
def logmod(x):
    return np.sign(x)*np.log(abs(x)+1)

In [ ]:
ltm = LTM("test",hrr_length, normalized)
ltm.print()

In [ ]:
t0 = time.time()

In [ ]:
non_obs = 0
current_atr = 0
current_wm = "I"

debug2 = False
step_store = []

for x in range(episodes):
        
    current_state = random.randint(0, size_of_maze - 1)
    start = current_state
    current_signal = np.random.choice(signals)
    
#     if num_non_obs_tasks == 0:
#         goal = goals[signals.index(current_signal)]
#     elif num_obs_tasks == 0:
#         if x%non_obs_task_switch_rate == 0:
#             non_obs = (non_obs+1)%num_non_obs_tasks
#         goal = goals[non_obs]
#     else:
#         if x%non_obs_task_switch_rate == 0:
#             non_obs = (non_obs+1)%num_non_obs_tasks
#         goal = goals[non_obs][signals.index(current_signal)]
        
    non_obs = (non_obs+1)%num_non_obs_tasks
    goal = goals[non_obs][signals.index(current_signal)]
        
    if goal == 0:
        goal_store[0][1] += 1
    if goal == 5:
        goal_store[1][1] += 1
    if goal == 10:
        goal_store[2][1] += 1
    if goal == 15:
        goal_store[3][1] += 1
        
    steps = 0
    opt_steps = get_opt_steps(current_state, goal, size_of_maze)
    
    # Reset trace
    eligibility *= 0.0
    
    if x > ((episodes*8) / 10) :
        debug2 = True
        rand_on = 0
        alpha = 0.0
    
    if debug:
        print("Goal: {0}, Signal: {1}, Non_Observable: {2}".format(goal, current_signal, non_obs))
        
    episode_memory = []
    for y in range(steps_till_quit):
        if (current_state == goal):
            encode_str = build_hrr_string(current_wm, current_signal, str(current_state) + "*rewardTkn", current_atr)
            goal_hrr = ltm.encode(encode_str)
            goal_value = np.dot(weights, goal_hrr) + bias
            episode_memory += [[current_state, current_value, goal]]

            error = reward_good - goal_value
            eligibility *= eli_lambda
            eligibility = eligibility + goal_hrr
            weights = np.add(weights, (alpha * logmod(error) * eligibility))
            
            if(debug):
                print("In goal with value {0}".format(goal_value))
            
            break
            
        
        # Store info about previous state      
        previous_wm = current_wm
        previous_signal = current_signal
        previous_state = current_state
        previous_atr = current_atr
        
        if debug:
            print("Previous WM:, {0}, Signal:, {1}, State, {2}, ATR:, {3}".format(previous_wm, previous_signal, previous_state, previous_atr))
        
        encode_str = build_hrr_string(previous_wm, previous_signal, previous_state, previous_atr)
        previous_state_hrr = ltm.encode(encode_str)
        previous_value = np.dot(weights, previous_state_hrr) + bias
        
        if debug:
            print("Started with state: {0}, State Value: {1}, WM: {2},  Atr: {3}".format(previous_state, previous_value, previous_wm, previous_atr))
            
        current_signal = "I"
        left, right = get_moves(previous_state, size_of_maze)
        if previous_signal != "I":
            previous_signal += "In"

        # Make the move
        move, wm, atr, random_move = move_policy(goal, [left, right], [previous_wm, previous_signal], [current_signal], previous_atr, rand_on)
        steps += 1
        
        current_wm = wm
        current_state = move
        current_atr = atr
        
        if random_move:
            eligibility *= 0.0
            
        if(debug):
            print("Moves {0}, taken {1}".format([left, right], move))
            
        if debug:
            print("Current WM {0}, Current Signal {1}, Current state {2}, Current ATR {3}".format(current_wm, current_signal, current_state, current_atr))
            
        if current_state == goal:
            encode_str = build_hrr_string(current_wm, current_signal, str(current_state) + "*rewardTkn", current_atr)     
            if debug:
                print("In goal: WM: {1}, ATR: {2}".format(current_wm, current_atr))
        else:
            encode_str = build_hrr_string(current_wm, current_signal, current_state, current_atr)
            
        current_state_hrr = ltm.encode(encode_str)
        current_value = np.dot(weights, current_state_hrr) + bias
        
        sarsa_error = (reward_bad + discount * current_value) - previous_value
        eligibility *= eli_lambda
        eligibility = eligibility + previous_state_hrr
        weights = np.add(weights, (alpha * logmod(sarsa_error) * eligibility))
        if sarsa_error > treshold_positive and num_non_obs_tasks > 0:
            current_atr = context_policy_positive(current_wm, current_signal, current_state, current_atr)
            eligibility *= 0.0
            steps = 0
            start = current_state
            opt_steps = get_opt_steps(current_state, goal, size_of_maze)
            if(debug):
                print("Changed atr from {0} to {1}".format(previous_atr, current_atr))
        if sarsa_error < threshold_negative and num_non_obs_tasks > 0:
            current_atr = context_policy_negative(previous_atr)
            eligibility *= 0.0
            steps = 0
            start = current_state
            opt_steps = get_opt_steps(current_state, goal, size_of_maze)
            if(debug):
                print("Changed atr from {0} to {1}".format(previous_atr, current_atr))
        
        if debug:
            input("")
    
    if debug2:
        if current_state == goal:
            step_store += [steps - opt_steps]
        else:
            step_store += [1000]

    update_progress(x / episodes, x)

update_progress(1, episodes)

In [ ]:
plt.plot(step_store)
plt.show()
accuracy = (len(step_store)-np.count_nonzero(step_store))*100.0 / len(step_store)
print(accuracy)

In [ ]:
fig, axes = plt.subplots(nrows=num_non_obs_tasks * 2, ncols=num_obs_tasks+1)
fig.set_figwidth(20)
fig.set_figheight(20)

for x in range(num_non_obs_tasks):
    
    x_ind = x * 2
    
    y_for_rwd = 0
    y_for_no_rwd = 0
    
    for wm in list(dict.fromkeys([signal + "In" if signal != "I" else signal for signal in signals] + ["I"])):
        position = np.arange(size_of_maze)
        value = np.zeros(size_of_maze)
        for signal in signals + ["I"]:
            lab = "WM:" + wm + "*Signal:" + signal + "*rewardTkn*Atr:" + str(x)
            for state in range(size_of_maze):
                encode_str = build_hrr_string(wm, signal, str(state) + "*rewardTkn", x)
                value[state] = np.dot(weights, ltm.encode(encode_str)) + bias
            axes[x_ind,y_for_rwd].title.set_text(wm + " with rewardTkn " + "Atr: " + str(x))
#             axes[x_ind,y_for_rwd].set_ylim([-0.2, 1.2])
            axes[x_ind,y_for_rwd].plot(position, value, label=lab)
            
            axes[x_ind,y_for_rwd].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
                  fancybox=True, shadow=True, ncol=1, prop={'size': 10})
        y_for_rwd += 1
        
        y = x + 1
        value = np.zeros(size_of_maze)
        for signal in list(dict.fromkeys(signals + ["I"])):
            lab = "WM:" + wm + "*Signal:" + signal + "*Atr:" + str(x)
            for state in range(size_of_maze):
                encode_str = build_hrr_string(wm, signal, str(state), x)
                value[state] = np.dot(weights, ltm.encode(encode_str)) + bias
            axes[x_ind+1,y_for_no_rwd].title.set_text(wm + " Atr: " + str(x))
#             axes[x_ind+1,y_for_no_rwd].set_ylim([-0.2, 1.2])
            axes[x_ind+1,y_for_no_rwd].plot(position, value, label=lab)
            axes[x_ind+1,y_for_no_rwd].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
                  fancybox=True, shadow=True, ncol=1, prop={'size': 10})
        y_for_no_rwd += 1

plt.suptitle("{0} Non-Observable tasks and {1} Observable tasks with goals: {2}".format(num_non_obs_tasks, num_obs_tasks, goals), fontsize=30)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.savefig("{0}.png".format(accuracy))
plt.show()

In [ ]:
t1 = time.time()
print(t1-t0)

In [ ]:
goal_store

In [ ]:
# plt.savefig("{0}2.png".format(accuracy))

In [ ]:
ltm.count()